In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Load cleaned data
data = pd.read_csv("../../data/cleaned/data_train_cleaned.csv")

# Split data into features (X) and target (y)
X = data.drop("Status", axis=1)
y = data["Status"]

In [3]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Create a random forest classifier with 100 trees
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.DataFrame(
    clf.feature_importances_, index=X.columns, columns=["Importance"]
)

# Print the most important features
print(feature_importances)

           0
0   0.014273
1   0.009919
2   0.001219
3   0.036971
4   0.006648
5   0.002736
6   0.005597
7   0.005603
8   0.006616
9   0.005274
10  0.002286
11  0.001826
12  0.003155
13  0.024833
14  0.012722
15  0.016572
16  0.008753
17  0.006509
18  0.024411
19  0.013790
20  0.018624
21  0.398434
22  0.373229
